In [28]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import onnx
import keras2onnx
import random
from es import SimpleGA, OpenES

In [16]:
input_shape = (15,15,2)
input_len = input_shape[0] * input_shape[1] * input_shape[2]
model_name_onnx = "model4light.onnx"

In [17]:
"""
## Build the model
"""
print("input shape=" + str(input_shape))
"""model = keras.Sequential(
    [
        keras.Input(shape=input_len, name="input"),
        layers.Reshape(input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        #layers.Dropout(0.5),
        layers.Dense(10, activation="relu"),
        layers.Dense(1, activation="sigmoid", name="output")
    ]
)"""
"""
model = keras.Sequential()
model.add(layers.Dense(20, activation='sigmoid', input_shape = input_shape))
model.add(layers.Dense(20, activation='sigmoid'))
model.add(layers.Dense(num_classes, activation="softmax"))
"""

def create_model():
    return keras.Sequential(
        [
            keras.Input(shape=input_len, name="input"),
            layers.Reshape(input_shape),
            layers.Conv2D(16, kernel_size=(5, 5), activation="relu"),
            layers.Conv2D(8, kernel_size=(3, 3), activation="relu"),
            layers.Flatten(),
            #layers.Dropout(0.5),
            layers.Dense(2, activation="relu"),
            layers.Dense(1, activation="sigmoid", name="output")
        ]
    )
model = create_model()

model.summary()

input shape=(15, 15, 2)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 15, 15, 2)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 16)        816       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 8)           1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 648)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1298      
_________________________________________________________________
output (Dense)               (None, 1)                 3         
Total params: 3,277
Trainable params: 3,277
Non-trainable params: 0
____________________________

In [18]:
model.compile()

In [61]:
onnx_model = keras2onnx.convert_keras(model, model.name)

onnx.save_model(onnx_model, model_name_onnx)

In [113]:
random.seed(1)
field_count = 25
random_fields = np.zeros((field_count,input_shape[0],input_shape[1],input_shape[2]), dtype="float32")
print(random_fields.shape)
for field in range(field_count):
    for row in range(input_shape[0]):
        for col in range(input_shape[1]):
            r = random.randint(0,3);
            if(r == 0 or r == 1):
                random_fields[field][row][col][r] = 1

random_fields_flat = random_fields.reshape((field_count,input_len))
#print(random_fields_flat.shape)
#for field in random_fields_flat:
#    print(field)

print(model.predict(random_fields_flat,verbose=1))

(25, 15, 15, 2)
1/1 [==============================] - 0s 988us/step
[[0.48035207]
 [0.48522398]
 [0.49998918]
 [0.49204195]
 [0.47665578]
 [0.48409656]
 [0.484397  ]
 [0.48887563]
 [0.47591773]
 [0.4936734 ]
 [0.48398066]
 [0.47898668]
 [0.5006399 ]
 [0.49423417]
 [0.48157942]
 [0.47734782]
 [0.5091617 ]
 [0.4840211 ]
 [0.49633208]
 [0.49159396]
 [0.48231378]
 [0.47738966]
 [0.47880867]
 [0.49641684]
 [0.48078114]]


In [10]:
model_weights = model.get_weights()
### unwrap layers weights
layer_count = len(model_weights)
print("layer count: " + str(layer_count))
layer_shapes = [None] * layer_count
flat_layers = [None] * layer_count
layer_n = 0;
for layer in model_weights:
    np_layer = np.array(layer)
    layer_shapes[layer_n] = np_layer.shape
    flat_layers[layer_n] = np_layer.reshape(-1)
    layer_n += 1

    
print("layer shapes: " + str(layer_shapes))
print("layer sizes: ");
for layer in model_weights:
    print(layer.size)
    
flat_weights = np.concatenate(flat_layers)
print("weight count: " + str(len(flat_weights)))
print("src weights:")
for i in range(20):
    print(flat_weights[i])

layer count: 8
layer shapes: [(5, 5, 2, 16), (16,), (3, 3, 16, 8), (8,), (648, 2), (2,), (2, 1), (1,)]
layer sizes: 
800
16
1152
8
1296
2
2
1
weight count: 3277
src weights:
0.055121228
0.049297348
0.10106319
0.11255874
0.06242843
0.08430731
0.0069460943
0.08494167
-0.05940278
-0.021465987
-0.095588066
-0.083381586
-0.112503506
0.05981587
-0.06991546
-0.02762466
-0.015926719
-0.08979946
0.08021371
-0.017605126


In [11]:
##### manipulate layer weights

for i in range(1, len(flat_weights), 1):
    flat_weights[i] = 0
    
print("manipulated weights:")
for i in range(20):
    print(flat_weights[i])
    

manipulated weights:
0.055121228
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [12]:
#### wrap layer weights back
split_indexes = [len(l) for l in flat_layers]
print(split_indexes)
for i in range(1, len(split_indexes)):
    split_indexes[i] += split_indexes[i-1]
print(split_indexes)
flat_layers = np.split(flat_weights, split_indexes)
flat_layers.pop()
layer_n = 0
for layer in flat_layers:
    model_weights[layer_n] = layer.reshape(layer_shapes[layer_n])
    layer_n += 1

model.set_weights(model_weights)

[800, 16, 1152, 8, 1296, 2, 2, 1]
[800, 816, 1968, 1976, 3272, 3274, 3276, 3277]


In [105]:
NPARAMS = sum([np.array(layer).size for layer in model.get_weights()]) # make this a n-dimensinal problem.
print("NPARAMS: " + str(NPARAMS))
NPOPULATION = 16 # population size .
MAX_ITERATION = 40 # run each solver for n generations.

NPARAMS: 3277


In [106]:
import gomokutrainer
import importlib
importlib.reload(gomokutrainer)
evaluator = gomokutrainer.Evaluator(create_model, "../EstimatorExe/OnnxEstimator.exe")
oes = OpenES(NPARAMS,                  # number of model parameters
            sigma_init=0.5,            # initial standard deviation
            sigma_decay=0.999,         # don't anneal standard deviation
            learning_rate=0.1,         # learning rate for standard deviation
            learning_rate_decay = 1.0, # annealing the learning rate
            popsize=NPOPULATION,       # population size
            antithetic=False,          # whether to use antithetic sampling
            weight_decay=0.00,         # weight decay coefficient
            rank_fitness=False,        # use rank rather than fitness numbers
            forget_best=False)
list_of_flat_weights = oes.ask()
print("list of flat weights len: " + str(len(list_of_flat_weights)))
eval_result = evaluator.evaluate(list_of_flat_weights, "C:\Projects\Testing")
print("evaluation result: " + str(eval_result))



tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14


list of flat weights len: 16


The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ON

starting evaluator exe
C:\Users\User\Documents\code\RTU\BakD\GomokuTrainer\EstimatorExe\OnnxEstimator.exe C:\Projects\Testing
0
evaluator exe finished
evaluation result: [1, 1, 0, 4, 0, 0, 2, 0, 1, 0, 0, 2, 3, 0, 1, 0]


In [116]:
def test_solver(solver):
    history = []
    for j in range(MAX_ITERATION):
        list_of_flat_weights = solver.ask()
        eval_result = evaluator.evaluate(list_of_flat_weights, "C:\Projects\Testing2")
        solver.tell(eval_result)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
    return history

In [117]:
oes = OpenES(NPARAMS,                  # number of model parameters
            sigma_init=0.5,            # initial standard deviation
            sigma_decay=0.999,         # don't anneal standard deviation
            learning_rate=0.1,         # learning rate for standard deviation
            learning_rate_decay = 1.0, # annealing the learning rate
            popsize=NPOPULATION,       # population size
            antithetic=False,          # whether to use antithetic sampling
            weight_decay=0.00,         # weight decay coefficient
            rank_fitness=True,        # use rank rather than fitness numbers
            forget_best=True)

In [119]:
oes_history = test_solver(oes)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -

starting evaluator exe
C:\Users\User\Documents\code\RTU\BakD\GomokuTrainer\EstimatorExe\OnnxEstimator.exe C:\Projects\Testing2


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False


0
evaluator exe finished


The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf exec

...

starting evaluator exe
C:\Users\User\Documents\code\RTU\BakD\GomokuTrainer\EstimatorExe\OnnxEstimator.exe C:\Projects\Testing2


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False


0
evaluator exe finished


The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 14
The maximum opset needed by this model is only 11.
tf exec

starting evaluator exe
C:\Users\User\Documents\code\RTU\BakD\GomokuTrainer\EstimatorExe\OnnxEstimator.exe C:\Projects\Testing2
0
evaluator exe finished


In [ ]:
ga = SimpleGA(NPARAMS,                # number of model parameters
               sigma_init=0.5,        # initial standard deviation
               popsize=NPOPULATION,   # population size
               elite_ratio=0.1,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               weight_decay=0.00,     # weight decay coefficient
              )

In [ ]:
ga_history = test_solver(ga)